In [1]:
import numpy as np
import CDPL.Chem as Chem
import CDPL.Biomol as Biomol
import CDPL.Math as Math
import CDPL.Pharm as Pharm
import CDPL.Base as Base
import CDPL.Shape as Shape
import sys
sys.path.append('../utils')
from Protein import Protein
from ProteinTools import THREE_LETTER_AMINO_ACID_CODES
from scipy.spatial.transform import Rotation
from MoleculeTools import clean, sanitize_mol

In [2]:
path = '../Data/benchmark5.5/structures/'

In [24]:
p = Protein()
p.fromFile('{}1A2K_r_u.pdb'.format(path))

In [25]:
p.numAtoms

1979

In [26]:
# remove ligands and other crystalization artifacts
p.separateLigandFromProtein()

In [27]:
p.numAtoms

1979

In [23]:
sanitized = sanitize_mol(p, makeHydrogenComplete=True)
sanitized.numAtoms

3904

In [8]:
p.numAtoms

3904

In [9]:
Pharm.prepareForPharmacophoreGeneration(p)

In [10]:
Chem.generateHydrogen3DCoordinates(p, True)

In [11]:
surfaceExtractor = Chem.SurfaceAtomExtractor()

In [28]:
fragment = Chem.Fragment()

In [29]:
surfaceExtractor.extract(p, p, fragment)

In [30]:
fragment.numAtoms

1017

In [31]:
for a in fragment.atoms: 
    print(Chem.getElementName(a))

Nitrogen
Carbon
Carbon
Oxygen
Nitrogen
Carbon
Carbon
Oxygen
Carbon
Carbon
Oxygen
Oxygen
Nitrogen
Carbon
Oxygen
Carbon
Carbon
Carbon
Carbon
Nitrogen
Carbon
Carbon
Carbon
Carbon
Nitrogen
Carbon
Carbon
Carbon
Carbon
Carbon
Nitrogen
Carbon
Carbon
Carbon
Carbon
Oxygen
Carbon
Carbon
Carbon
Oxygen
Oxygen
Carbon
Oxygen
Carbon
Carbon
Carbon
Oxygen
Nitrogen
Oxygen
Carbon
Carbon
Carbon
Carbon
Oxygen
Carbon
Oxygen
Nitrogen
Carbon
Oxygen
Carbon
Oxygen
Nitrogen
Carbon
Carbon
Carbon
Carbon
Carbon
Carbon
Carbon
Carbon
Carbon
Carbon
Carbon
Oxygen
Nitrogen
Carbon
Carbon
Carbon
Nitrogen
Carbon
Carbon
Nitrogen
Carbon
Carbon
Oxygen
Carbon
Carbon
Carbon
Oxygen
Carbon
Carbon
Carbon
Oxygen
Nitrogen
Carbon
Carbon
Carbon
Carbon
Oxygen
Carbon
Carbon
Carbon
Oxygen
Carbon
Oxygen
Carbon
Carbon
Oxygen
Carbon
Carbon
Oxygen
Nitrogen
Nitrogen
Carbon
Carbon
Carbon
Oxygen
Oxygen
Nitrogen
Oxygen
Carbon
Carbon
Nitrogen
Nitrogen
Oxygen
Carbon
Oxygen
Carbon
Carbon
Oxygen
Carbon
Carbon
Carbon
Oxygen
Nitrogen
Carbon
Carbon
Car

In [11]:
def getShape(mol, scaleFactor=1):
    shape = Shape.GaussianShape()
    shapeFunc = Shape.GaussianShapeFunction()
    Shape.generateGaussianShape(mol, shape, inc_h=True)
    for e in shape: 
        e.setRadius(e.getRadius()*scaleFactor)
    shapeFunc.setMaxOrder(6)
    shapeFunc.setShape(shape)
    return shape, shapeFunc

In [12]:
def getShapeWithIncreasedRadius(mol, increase=0.5):
    shape = Shape.GaussianShape()
    shapeFunc = Shape.GaussianShapeFunction()
    Shape.generateGaussianShape(mol, shape, inc_h=True)
    for e in shape: 
        e.setRadius(e.getRadius()+increase)
    shapeFunc.setMaxOrder(6)
    shapeFunc.setShape(shape)
    return shape, shapeFunc

In [18]:
for i in range(6): 
    scale = 1+i/10
    shape, shapeFunc = getShape(p, scaleFactor=scale)
    print(scale, shapeFunc.surfaceArea)

1.0 27988.12049910137
1.1 28410.142447759194
1.2 27059.701115325683
1.3 20202.99639539608
1.4 -15510.980247224861
1.5 -195262.04742137715


In [14]:
for i in range(10): 
    scale = i/10
    shape, shapeFunc = getShapeWithIncreasedRadius(p, increase=scale)
    print(scale, shapeFunc.surfaceArea)

0.0 27988.12049910137
0.1 28441.0706365565
0.2 28247.046684722096
0.3 26727.531577847974
0.4 22805.401993716994
0.5 12077.466539929597
0.6 -22882.477273865916
0.7 -134351.87936788626
0.8 -422667.36782250006
0.9 -1122910.2450750417


In [26]:
def calcSurfaceAtoms(mol):
    surfaceAtoms = []
    for a in mol.atoms:
        surfaceContribution = shapeFunc.calcSurfaceArea(mol.getAtomIndex(a))
        if surfaceContribution > 0: 
            print('Surface contribution:', surfaceContribution, '\tAtom Type: ', Chem.getElementName(a))
            surfaceAtoms.append(a)
    return surfaceAtoms 

In [27]:
surfaceAtoms = calcSurfaceAtoms(p)
len(surfaceAtoms)

Surface contribution: 12.34271100725867 	Atom Type:  Nitrogen
Surface contribution: 13.504139636180797 	Atom Type:  Carbon
Surface contribution: 9.662774780451937 	Atom Type:  Carbon
Surface contribution: 18.10283150242904 	Atom Type:  Oxygen
Surface contribution: 5.546846297317168 	Atom Type:  Nitrogen
Surface contribution: 6.420033451761523 	Atom Type:  Carbon
Surface contribution: 8.522328203769776 	Atom Type:  Carbon
Surface contribution: 17.82148752213031 	Atom Type:  Oxygen
Surface contribution: 8.431764478571838 	Atom Type:  Carbon
Surface contribution: 11.875283127518117 	Atom Type:  Carbon
Surface contribution: 19.780070769076683 	Atom Type:  Oxygen
Surface contribution: 16.691455740524155 	Atom Type:  Oxygen
Surface contribution: 5.441310690396132 	Atom Type:  Nitrogen
Surface contribution: 5.222562691170674 	Atom Type:  Carbon
Surface contribution: 5.011309803382247 	Atom Type:  Carbon
Surface contribution: 10.385893399796776 	Atom Type:  Oxygen
Surface contribution: 8.08011

Surface contribution: 10.249049658555428 	Atom Type:  Carbon
Surface contribution: 7.13097655005979 	Atom Type:  Carbon
Surface contribution: 9.611665494975005 	Atom Type:  Carbon
Surface contribution: 7.214382017084476 	Atom Type:  Carbon
Surface contribution: 10.654857898355344 	Atom Type:  Oxygen
Surface contribution: 1.4867209966601718 	Atom Type:  Nitrogen
Surface contribution: 3.5578571067384854 	Atom Type:  Carbon
Surface contribution: 2.3410929990689406 	Atom Type:  Carbon
Surface contribution: 5.056397596272665 	Atom Type:  Oxygen
Surface contribution: 7.691319136336909 	Atom Type:  Carbon
Surface contribution: 7.657145546692291 	Atom Type:  Carbon
Surface contribution: 5.551808788413459 	Atom Type:  Carbon
Surface contribution: 7.6701641586406915 	Atom Type:  Carbon
Surface contribution: 7.101075835878861 	Atom Type:  Carbon
Surface contribution: 9.272917661260852 	Atom Type:  Carbon
Surface contribution: 9.395433895667637 	Atom Type:  Carbon
Surface contribution: 8.951070298

Surface contribution: 7.881064394411256 	Atom Type:  Carbon
Surface contribution: 6.7611487376206725 	Atom Type:  Carbon
Surface contribution: 17.229349977563476 	Atom Type:  Oxygen
Surface contribution: 14.478938317101319 	Atom Type:  Carbon
Surface contribution: 3.2414770726056235 	Atom Type:  Nitrogen
Surface contribution: 5.031416971586171 	Atom Type:  Carbon
Surface contribution: 4.294202875054445 	Atom Type:  Carbon
Surface contribution: 8.831979483997648 	Atom Type:  Oxygen
Surface contribution: 4.8848439543699325 	Atom Type:  Carbon
Surface contribution: 4.058071152896437 	Atom Type:  Oxygen
Surface contribution: 3.632034548603845 	Atom Type:  Nitrogen
Surface contribution: 4.690336628830406 	Atom Type:  Carbon
Surface contribution: 4.960857678857277 	Atom Type:  Carbon
Surface contribution: 8.666917339481904 	Atom Type:  Oxygen
Surface contribution: 7.8122394684796514 	Atom Type:  Carbon
Surface contribution: 28.130962048859022 	Atom Type:  Sulfur
Surface contribution: 4.57573

Surface contribution: 7.069975424667782 	Atom Type:  Carbon
Surface contribution: 7.020818580629313 	Atom Type:  Carbon
Surface contribution: 11.501006994735148 	Atom Type:  Carbon
Surface contribution: 10.927305333069263 	Atom Type:  Carbon
Surface contribution: 2.01611955800752 	Atom Type:  Nitrogen
Surface contribution: 6.094581680421886 	Atom Type:  Carbon
Surface contribution: 5.295398873120048 	Atom Type:  Carbon
Surface contribution: 16.78585104340614 	Atom Type:  Oxygen
Surface contribution: 10.106343805314545 	Atom Type:  Carbon
Surface contribution: 14.213385742702913 	Atom Type:  Oxygen
Surface contribution: 2.570002418254691 	Atom Type:  Nitrogen
Surface contribution: 6.543840038148458 	Atom Type:  Carbon
Surface contribution: 6.0661024680247895 	Atom Type:  Carbon
Surface contribution: 16.25289205959282 	Atom Type:  Oxygen
Surface contribution: 10.216629418405912 	Atom Type:  Carbon
Surface contribution: 10.902856032521697 	Atom Type:  Oxygen
Surface contribution: 2.434107

Surface contribution: 3.206814758072566 	Atom Type:  Nitrogen
Surface contribution: 5.846295395870611 	Atom Type:  Carbon
Surface contribution: 4.6737356357247055 	Atom Type:  Carbon
Surface contribution: 13.082607628470138 	Atom Type:  Oxygen
Surface contribution: 9.641239596581022 	Atom Type:  Carbon
Surface contribution: 9.049711410222239 	Atom Type:  Carbon
Surface contribution: 6.080992321645905 	Atom Type:  Carbon
Surface contribution: 3.348900301397738 	Atom Type:  Nitrogen
Surface contribution: 4.3500620869585624 	Atom Type:  Carbon
Surface contribution: 4.044532236079016 	Atom Type:  Carbon
Surface contribution: 11.339807204558948 	Atom Type:  Oxygen
Surface contribution: 5.287898718411562 	Atom Type:  Carbon
Surface contribution: 5.408348349212081 	Atom Type:  Oxygen
Surface contribution: 6.912142852838837 	Atom Type:  Carbon
Surface contribution: 2.890703346727203 	Atom Type:  Nitrogen
Surface contribution: 6.076458785186039 	Atom Type:  Carbon
Surface contribution: 5.290527

...

Surface contribution: 7.135507187937195 	Atom Type:  Hydrogen
Surface contribution: 0.965513230562828 	Atom Type:  Hydrogen
Surface contribution: 5.552342789866859 	Atom Type:  Hydrogen
Surface contribution: 7.197334971455741 	Atom Type:  Hydrogen
Surface contribution: 9.244726593680817 	Atom Type:  Hydrogen
Surface contribution: 8.356891058473408 	Atom Type:  Hydrogen
Surface contribution: 6.40130654646037 	Atom Type:  Hydrogen
Surface contribution: 7.773729399381608 	Atom Type:  Hydrogen
Surface contribution: 4.85642453644636 	Atom Type:  Hydrogen
Surface contribution: 7.3433394592628405 	Atom Type:  Hydrogen
Surface contribution: 2.2213357089747623 	Atom Type:  Hydrogen
Surface contribution: 5.826777761752934 	Atom Type:  Hydrogen
Surface contribution: 5.771560918692125 	Atom Type:  Hydrogen
Surface contribution: 6.7930808315723255 	Atom Type:  Hydrogen
Surface contribution: 5.673596129712682 	Atom Type:  Hydrogen
Surface contribution: 8.739170968479236 	Atom Type:  Hydrogen
Surface

Surface contribution: 5.558391767971074 	Atom Type:  Hydrogen
Surface contribution: 2.0815405771828948 	Atom Type:  Hydrogen
Surface contribution: 6.556289398870186 	Atom Type:  Hydrogen
Surface contribution: 6.489637895313401 	Atom Type:  Hydrogen
Surface contribution: 3.880967570583808 	Atom Type:  Hydrogen
Surface contribution: 8.066484716099286 	Atom Type:  Hydrogen
Surface contribution: 7.083260901581631 	Atom Type:  Hydrogen
Surface contribution: 2.507993078567478 	Atom Type:  Hydrogen
Surface contribution: 3.731720626201636 	Atom Type:  Hydrogen
Surface contribution: 5.141246995802841 	Atom Type:  Hydrogen
Surface contribution: 6.322452630441831 	Atom Type:  Hydrogen
Surface contribution: 6.923663689480673 	Atom Type:  Hydrogen
Surface contribution: 9.858051541689328 	Atom Type:  Hydrogen
Surface contribution: 11.226987430322525 	Atom Type:  Hydrogen
Surface contribution: 2.616942975286486 	Atom Type:  Hydrogen
Surface contribution: 5.001068586355359 	Atom Type:  Hydrogen
Surfac

Surface contribution: 7.972740199797391 	Atom Type:  Hydrogen
Surface contribution: 3.842915806706181 	Atom Type:  Hydrogen
Surface contribution: 4.797078207328507 	Atom Type:  Hydrogen
Surface contribution: 7.144177607314157 	Atom Type:  Hydrogen
Surface contribution: 8.421528014354697 	Atom Type:  Hydrogen
Surface contribution: 5.4750027868775 	Atom Type:  Hydrogen
Surface contribution: 4.034119008927117 	Atom Type:  Hydrogen
Surface contribution: 8.949857462744575 	Atom Type:  Hydrogen
Surface contribution: 9.355612468108523 	Atom Type:  Hydrogen
Surface contribution: 10.224773972720033 	Atom Type:  Hydrogen
Surface contribution: 7.565530275667193 	Atom Type:  Hydrogen
Surface contribution: 2.011715402748134 	Atom Type:  Hydrogen
Surface contribution: 6.050855239647821 	Atom Type:  Hydrogen
Surface contribution: 8.7265497900684 	Atom Type:  Hydrogen
Surface contribution: 6.594855853294926 	Atom Type:  Hydrogen
Surface contribution: 3.4563425552783533 	Atom Type:  Hydrogen
Surface co

Surface contribution: 5.893725541763526 	Atom Type:  Hydrogen
Surface contribution: 3.5971463198560265 	Atom Type:  Hydrogen
Surface contribution: 7.676031026184399 	Atom Type:  Hydrogen
Surface contribution: 8.541539730413444 	Atom Type:  Hydrogen
Surface contribution: 8.314623934197435 	Atom Type:  Hydrogen
Surface contribution: 3.995797723226678 	Atom Type:  Hydrogen
Surface contribution: 7.984213956828763 	Atom Type:  Hydrogen
Surface contribution: 6.83092990434048 	Atom Type:  Hydrogen
Surface contribution: 4.6598371540774295 	Atom Type:  Hydrogen
Surface contribution: 7.756029999483195 	Atom Type:  Hydrogen
Surface contribution: 6.589623121405962 	Atom Type:  Hydrogen
Surface contribution: 8.00938052669642 	Atom Type:  Hydrogen
Surface contribution: 5.955949480935909 	Atom Type:  Hydrogen
Surface contribution: 7.667745710747635 	Atom Type:  Hydrogen
Surface contribution: 11.257508622701895 	Atom Type:  Hydrogen
Surface contribution: 6.198886378672691 	Atom Type:  Hydrogen
Surface

3904

In [25]:
shape.getNumElements()

3904

In [11]:
surfaceExtractor = Chem.SurfaceAtomExtractor()

In [14]:
fragment = Chem.Fragment()

In [16]:
molGraph = Chem.MolecularGraph()

In [13]:
surfaceExtractor.extract(p)

ArgumentError: Python argument types in
    SurfaceAtomExtractor.extract(SurfaceAtomExtractor, Protein)
did not match C++ signature:
    extract(CDPL::Chem::SurfaceAtomExtractor {lvalue} self, CDPL::Chem::AtomContainer {lvalue} cntr, CDPL::Chem::MolecularGraph {lvalue} parent_molgraph, CDPL::Chem::Fragment {lvalue} frag)

In [15]:
Chem.SurfaceAtomExtractor()

In [17]:
molGraph = Chem.MolecularGraph()

ArgumentError: Python argument types in
    MolecularGraph.__init__(MolecularGraph, Protein)
did not match C++ signature:
    __init__(_object* self)

In [18]:
inputHandler = Chem.MolecularGraphInputHandler()

In [ ]:
inputHandler.createReader